# Aligning HST images to an absolute reference catalog
***

<div class="alert-danger">Note: The notebook in this repository 'Initializtion.ipynb' goes over many of the basic concepts such as the setup of the environment/package installation and should be read first if you are new to HST images, 'DrizzlePac' or 'Astroquery'.</div>

<div class="alert-warning">Note: This notebook is based on WFC3 ISR 2017-19: <a href="http://www.stsci.edu/hst/wfc3/documents/ISRs/WFC3-2017-19.pdf">Aligning HST Images to Gaia: a Faster Mosaicking Workflow</a> and contains a subset of the information/code found in <a href="https://github.com/spacetelescope/gaia_alignment">the repository here</a>.  For more information, see the notebook in that repository titled 'Gaia_alignment.ipynb'.</div>

## Introduction

The alignment of HST exposures is a critical step in image stacking/combination performed by software such as `AstroDrizzle`.  Generally, a relative alignment is performed that aligns one image (or multiple images) to another image which is designated as the reference image.  This makes it so the images are aligned to each other, but the pointing error of the observatory can still cause the images to have incorrect absolute astrometry.

When absolute astrometry is desired, the images can be aligned to an external catalog that is known to be on an absolute frame.  In this example, we will provide a workflow to query catalogs such as SDSS and Gaia via the astroquery package, and then align the images to that catalog via TweakReg.

For more information about TweakReg, see the other notebooks in this repository or the __[TweakReg Documentation](https://drizzlepac.readthedocs.io/en/deployment/tweakreg.html)__.

For more information on Astroquery, see the other notebooks in this repository or the __[Astroquery Documentation](https://astroquery.readthedocs.io/en/latest/)__.

In [1]:
import astropy.units as u
import glob
import numpy as np
import matplotlib.pyplot as plt
import os

from astropy.io import fits
from astropy.table import Table
from astropy.units import Quantity
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astroquery.mast import Observations
from astroquery.sdss import SDSS

#from ccdproc import ImageFileCollection
from IPython.display import Image

from drizzlepac import tweakreg
from drizzlepac import astrodrizzle

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


# 1. Set up the data

In [3]:
from utils import header_utils as hutils

In [4]:
from pathlib import Path
Path("/user/jaguilar/tr14/code/utils/header_utils.py").parent / "../../data/tables/"

PosixPath('/user/jaguilar/tr14/code/utils/../../data/tables')

In [5]:
from importlib import reload
reload(hutils)

<module 'utils.header_utils' from '/user/jaguilar/tr14/code/utils/header_utils.py'>

In [13]:
reload(hutils)
prihdrs = hutils.load_headers('pri')
scihdrs = hutils.load_headers('sci')

In [9]:
import pandas as pd

In [14]:
# TH.print_columns(prihdrs)
gb_filter = prihdrs.groupby('filter')
gb_filter

In [ ]:
TH.print_columns(prihdrs)

In [ ]:
# re-download the data from MAST, following the example
proposids = prihdrs['proposid'].unique()
filters = prihdrs['filter'].unique()
prihdrs.query(f"proposid == {proposids[0]}")

### Inspect the image header

The cell below shows how to query information from the image header using `ImageFileCollection` in `ccdproc`. 
We see that the 1st exposure is 30 seconds and the 2nd and 3rd exposures are 360 seconds. The 3rd exposure is dithered by ~82" in the Y-direction which is approximately the width of one UVIS chip. 

# 2. Querying catalogs

Now that we have the images, we will download the reference catalogs from both SDSS and Gaia using `astroquery`.

### 2a. Identify Coordinates
We will first create a SkyCoord Object to point astroquery to where we are looking on the sky.  Since our example uses data from NGC 6791, we will use the `ra_targ` and `dec_targ` keywords from the first image to get the coordinates of the object.

In [ ]:
RA, Dec = prihdrs[['ra_targ','dec_targ']].mean()

### 2b. SDSS Query
We now give those values to an astropy `SkyCoord` object, which we will pass to the SDSS.  Additionally, we use an astropy `Quantity` object to create a radius for the SDSS query.  We set the radius to 6 arcminutes to comfortably cover the area of our images. For reference UVIS detector field of view is ~2.7'x2.7' and a y-dither of 82" covers a total area on the sky of ~2.7'x4.1'.

In [ ]:
coord = SkyCoord(ra=RA, dec=Dec, unit=(u.deg, u.deg))
radius = Quantity(6., u.arcmin)

Then we only need to perform the query via the `SDSS.query_region` method of `astroquery.sdss`. The `spectro=False` keyword argument means we want to exclude spectroscopic objects, as we are looking for objects to match with an image.  

In the fields parameter, we specify a list of fields we want returned by the query.  In this case we only need the position, and maybe a magnitude 'g' if we want to cut very dim and/or bright objects out of the catalog, as those are likely measured poorly. Details on selecting objects by magnitude may be found in the original ['Gaia_alignment' notebook](https://github.com/spacetelescope/gaia_alignment).  Many other fields are available in the SDSS query and are [documented here](http://cas.sdss.org/dr7/en/help/browser/description.asp?n=PhotoObj&t=V).

### 2c. Gaia Query
Similarly to SDSS, we can query Gaia catalogs for our target via `astroquery.gaia`.  We can use the same `coord` and  `radius` from the SDSS query.

In [ ]:
gaia_query = Gaia.query_object_async(coordinate=coord, radius=radius)
gaia_query

This query has returned very large number of columns. We want to pare down the catalog to make it easier to use with `TweakReg`.  
We can select only the useful columns via:

In [ ]:
reduced_query = gaia_query['ra', 'dec', 'phot_g_mean_mag']
reduced_query

Then we write this catalog to an ascii file for use with `TweakReg`.

In [ ]:
reduced_query.write('../data/align_catalog/gaia.cat', format='ascii.commented_header')

## 3. Running TweakReg
With the catalogs downloaded and the headers populated, we simply need to run TweakReg with each catalog passed into the `refcat` parameter. The steps below compare the astrometric residuals obtained from aligning to each `refcat`. In each test case, we set `updatehdr` to False until we are satisfied with the alignment by inspecting both the shift file and the astrometric residual plots.

As we can see, the RMS is fairly large at about 0.5 pixels, which is not a great fit. This is likely because the SDSS astrometric precision is not high enough to get good HST alignment. One approach would be to align the first image to SDSS and then align the remaining HST images to one another. This would improve both the absolute and relative alignment of the individual frames.

### 3b. Gaia Alignment

In [ ]:
from pathlib import Path
# [(Path("../data/tr14/") / i).as_posix() 
#  for i in gb_filter.get_group(filters[0])['FILENAME']]

In [ ]:
from utils import table_utils as tutils


In [ ]:
list_of_images = [(Path("../data/my_data/") / i).as_posix() 
                  #for i in gb_filter.get_group(filters[0]).query('SUBARRAY == False')['FILENAME']]
                  for i in tutils.get_value(prihdrs, 'subarray', False, 'filename')]
len(list_of_images)

In [ ]:
import matplotlib as mpl

In [ ]:
# hdulist = fits.open(list_of_images[0])
# data = hdulist[1].data
# fig, ax = plt.subplots(1,1)
# ax.imshow(data, norm=mpl.colors.LogNorm())
# hdulist.close()

In [ ]:
tweakreg.help()

In [ ]:
refcat = '../data/align_catalog/gaia.cat'
cw = 3.5  # Set to two times the FWHM of the PSF.
wcsname = 'Gaia'  # Specify the WCS name for this alignment
ff = list_of_images[:2]
thresh = 500.
tweakreg.TweakReg(ff,#'*flc.fits',  # Pass input images
                  updatehdr=False,  # update header with new WCS solution
                  imagefindcfg={'threshold':thresh,'conv_width':cw},  # Detection parameters, threshold varies for different data
                  refcat=refcat,  # Use user supplied catalog (Gaia)
                  interactive=False,
                  see2dplot=False,
                  shiftfile=True,  # Save out shift file (so we can look at shifts later)
                  outshifts='Gaia_shifts.txt',  # name of the shift file
                  wcsname=wcsname,  # Give our WCS a new name
                  reusename=True,
                  sigma=2.3,
                  ylimit=0.2,
                  fitgeometry='general')  # Use the 6 parameter fit
#%pdb 0
#%debug -b /Users/jaguilar/miniconda3/envs/orbitize/lib/python3.6/site-packages/drizzlepac/imgclasses.py:1604
# reload(tweakreg)
# tweakreg.TweakReg(list_of_images,
#                   refcat=refcat)

We can similarly look at the shift file from alignment to the Gaia catalog:

In [ ]:
for line in open('Gaia_shifts.txt').readlines():
    print(line)

In [ ]:
# Astrometric residual plots
# Image(filename='residuals_ibwb01xqq_flc.png',width=500, height=300)

In [ ]:
# Image(filename='residuals_ibwb01xrq_flc.png',width=500, height=300)

In [ ]:
# Image(filename='residuals_ibwb01xxq_flc.png',width=500, height=300)

As expected, the Gaia catalog does quite a bit better, with rms residuals less tha 0.05 pixels.  

To apply these transformations to the image, we simply need to run TweakReg the same as before, but set the parameter `updatehdr` equal to `True`:

In [ ]:
tweakreg.TweakReg(ff,  # Pass input images
                  updatehdr=True,  # update header with new WCS solution
                  imagefindcfg={'threshold': thresh, 'conv_width': cw},  # Detection parameters, threshold varies for different data
                  refcat=refcat,  # Use user supplied catalog (Gaia)
                  interactive=False,
                  see2dplot=False,
                  shiftfile=True,  # Save out shift file (so we can look at shifts later)
                  outshifts='Gaia_shifts.txt',  # name of the shift file
                  wcsname=wcsname,  # Give our WCS a new name
                  reusename=True,
                  sigma=2.3,
                  fitgeometry='general',  # Use the 6 parameter fit
                  clean=False,
                 )

## 4. Drizzle the Data

While the three sets of FLC files are now aligned, we drizzle together only the two long exposures. 

When exposures are very different lengths, drizzling them together doesn't work well when 'EXP' weighting is used. For objects that saturate in the long exposures, the problem occurs at the boundary where you transition from only short exposure to short plus long. Here the pixels getting power from long exposure pixels are only getting power from pixels whose centers are outside the ring, and thus they are weighted lower than they would be if they were getting values from both inside and outside the ring. The result is a discontinuity in the PSF radial profile and a resulting flux which is too low in those boundary pixels. For photometry of saturated objects, the short exposures should be drizzled separately from the long exposures. 

In [ ]:
# Look at the catalog

In [ ]:
cat_file = Path(f"./{Path(ff[0]).name.split('.')[0]}_catalog_fit.match")

In [ ]:
cat_file.exists()

In [ ]:
from utils import cat_utils
reload(cat_utils)

In [ ]:
cat_utils.cat_fit_columns

In [ ]:
import pandas as pd

In [ ]:
reload(cat_utils)

In [ ]:
master_catalog = pd.concat([cat_utils.catalog2pandas(f) 
                           for f in glob.glob("*catalog_fit.match")])
# make a unique index for each point source
master_catalog = master_catalog.reset_index(drop=True).reset_index()
master_catalog.rename(columns={'index':'sourceid'}, inplace=True)

In [ ]:
master_catalog.to_csv("../data/tables/master_fullframe_source_catalog.csv")

In [ ]:
files = master_catalog['file_id'].unique()

In [ ]:
files

In [ ]:
# example file
ff = files[0]
sub_catalog = master_catalog.query(f"file_id == '{ff}'")

In [ ]:
hdulist = fits.open(glob.glob(f"../data/my_data/{ff}*fits")[0])
data = hdulist[1].data
hdulist.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,15))
ax.imshow(data, origin='lower', norm=mpl.colors.LogNorm(), cmap=mpl.cm.magma)
ax.set_title(rf"Threshold = {thresh} $\sigma$ above local bgnd", size='xx-large')
ax.set_xlabel("X [pix]", size='xx-large')
ax.set_ylabel("Y [pix]", size='xx-large')
ax.tick_params(labelsize='x-large')
ax.scatter(sub_catalog['x_orig_input'], 
           sub_catalog['y_orig_input'], 
           marker='o', ec='w', fc='none', s=100)
ax.set_xlim(0, data.shape[1])
ax.set_ylim(0, data.shape[0])
# for i, row in sub_catalog.iterrows():
#     s = '{0:0.2f}, {1:0.2f}'.format(*row[['ra','dec']])
#     ax.annotate(s, row[['x_orig_input','y_orig_input']],
#                 fontsize='large',
#                 verticalalignment='top', horizontalalignment='center')
fname = f"./figs/tweakreg_demo_{thresh}.png"
print(fname)
if False:
    fig.savefig(fname, bbox_inches='tight')

In [ ]:
sub_catalog['input_id'].unique().size

In [ ]:
tab_html = master_catalog.to_html()

In [ ]:
html = open("table.html", 'w')
html.write(tab_html)
html.close()

In [ ]:
gb_fileid = master_catalog.groupby("file_id")

In [ ]:
groups = list(gb_fileid.groups.keys())

In [ ]:
print(gb_fileid.get_group(groups[0])['ref_id'].size)
print(gb_fileid.get_group(groups[1])['ref_id'].size)


In [ ]:
for i in cat1.ref_id.unique():
    if i not in cat0.ref_id.unique():
        print(i)

In [ ]:
cat0 = gb_fileid.get_group(groups[0])
cat1 = gb_fileid.get_group(groups[1])

In [ ]:
np.sort(cat0.ref_id.unique())

In [ ]:
np.sort(cat1.ref_id.unique())